In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import copy

* Data: aggregate infections / test consumption for all policies, all pooling methods, dilution only
* Format: pandas DataFrame

In [2]:
NUM_TRIALS = 10

BASE_PARAM_CONFIG = {
    "country": ["US"],
    "pop_size_default": 10000,
    "init_prev": [0.01],
    "init_prev_default": 0.01,
    "horizon_default": 100,
    "num_groups_default": 5,
    "pool_size": [10],
    "LoD_default": 1240,
    "LoD": [1240],
    "edge_weight_default": 10,
    "edge_weight": [10],
    "alpha_default": 5.0,
    "alpha": [2.0],
    "peak_VL": [6.0],
    "distancing_scale": [50.0],
    "beta_default": 0.1,  # transmissibility
    "sigma_default": 0.2,  # rate E --> I_pre
    "lamda_default": 0.5,  # rate I_pre --> I_(a)sym
    "gamma_default": 0.25,  # rate I_sym --> R
}

In [3]:
import pickle

param_config = copy.deepcopy(BASE_PARAM_CONFIG)

param_config["num_groups"] = [5]
param_config["dilute"] = ["average"]

In [4]:
import itertools

param_values = {}

for param in [
    "country",
    "pop_size",
    "init_prev",
    "num_groups",
    "pool_size",
    "horizon",
    "beta",
    "sigma",
    "lamda",
    "gamma",
    "LoD",
    "edge_weight",
    "alpha",
    "peak_VL",
    "distancing_scale",
    "dilute",
]:
    if param in param_config:
        param_values[param] = param_config[param]
    else:
        param_values[param] = [param_config[param + "_default"]]

all_param_configs = [
    dict(zip(param_values.keys(), x)) for x in itertools.product(*param_values.values())
]

In [5]:
import copy
from collections import defaultdict

all_data_dilution = []

for param_config_single in all_param_configs:

    if param_config_single["dilute"] != "average":
        continue

    traj_info = [param_config_single, {}]

    path = f"../data/raw/{param_config_single['country']}"
    for param in [
        "pop_size",
        "init_prev",
        "num_groups",
        "pool_size",
        "horizon",
        "beta",
        "sigma",
        "lamda",
        "gamma",
        "LoD",
        "edge_weight",
        "alpha",
        "peak_VL",
        "distancing_scale",
        "dilute",
    ]:
        path += f"_{param}={param_config_single[param]}"

    data = copy.deepcopy(param_config_single)

    results = defaultdict(list)

    for pooling_strategy in ["naive", "correlated", "correlated_weak"]:
        for i in range(1, NUM_TRIALS + 1):
            filepath = path + f"/{pooling_strategy}/results_{i}.pickle"
            print(f"loading {filepath}")
            try:
                with open(filepath, "rb") as f:
                    results[pooling_strategy].append(pickle.load(f))
            except Exception as e:
                print("error: ", e)
                continue

    if len(results["naive"]) == 0:
        continue

    for metric in ["cumInfections", "cum_num_tests"]:

        for pooling_strategy in ["naive", "correlated", "correlated_weak"]:

            results_tmp = []
            for SEED in range(1, NUM_TRIALS + 1):
                try:
                    results_tmp.append(
                        [x[metric] for x in results[pooling_strategy][SEED]]
                    )
                except:
                    pass

            results_tmp_ = np.array(
                [(xi + [xi[-1]] * (100 - len(xi)))[:100] for xi in results_tmp],
                dtype=float,
            )
            mean = np.nanmean(results_tmp_, axis=0)
            sem = np.nanstd(results_tmp_, axis=0) / np.sqrt(NUM_TRIALS)
            traj_info[1][metric + "_" + pooling_strategy] = [mean, sem]  # TODO: confirm

            if metric in ["cumInfections", "cum_num_tests"]:
                data[f"{metric}_{pooling_strategy}_mean"] = mean[-1]
                data[f"{metric}_{pooling_strategy}_sem"] = sem[-1]
            else:
                data[f"{metric}_{pooling_strategy}_mean"] = np.nanmean(mean)
                data[f"{metric}_{pooling_strategy}_sem"] = np.nanmean(sem)

    all_data_dilution.append(data)

loading ../data/raw/US_pop_size=10000_init_prev=0.01_num_groups=5_pool_size=10_horizon=100_beta=0.1_sigma=0.2_lamda=0.5_gamma=0.25_LoD=1240_edge_weight=10_alpha=2.0_peak_VL=6.0_distancing_scale=50.0_dilute=average/naive/results_1.pickle
loading ../data/raw/US_pop_size=10000_init_prev=0.01_num_groups=5_pool_size=10_horizon=100_beta=0.1_sigma=0.2_lamda=0.5_gamma=0.25_LoD=1240_edge_weight=10_alpha=2.0_peak_VL=6.0_distancing_scale=50.0_dilute=average/naive/results_2.pickle
loading ../data/raw/US_pop_size=10000_init_prev=0.01_num_groups=5_pool_size=10_horizon=100_beta=0.1_sigma=0.2_lamda=0.5_gamma=0.25_LoD=1240_edge_weight=10_alpha=2.0_peak_VL=6.0_distancing_scale=50.0_dilute=average/naive/results_3.pickle
loading ../data/raw/US_pop_size=10000_init_prev=0.01_num_groups=5_pool_size=10_horizon=100_beta=0.1_sigma=0.2_lamda=0.5_gamma=0.25_LoD=1240_edge_weight=10_alpha=2.0_peak_VL=6.0_distancing_scale=50.0_dilute=average/naive/results_4.pickle
loading ../data/raw/US_pop_size=10000_init_prev=0.01

In [6]:
df = pd.DataFrame(all_data_dilution)
df = df[
    (df["init_prev"] == 0.01)
    & (df["distancing_scale"] == 50)
    & (df["alpha"] == 2.0)
    & (df["dilute"] == "average")
]

In [7]:
df.drop(
    columns=[
        "country",
        "pop_size",
        "init_prev",
        "horizon",
        "beta",
        "sigma",
        "lamda",
        "gamma",
        "LoD",
        "edge_weight",
        "alpha",
        "peak_VL",
        "distancing_scale",
        "dilute",
    ],
    inplace=True,
)

In [8]:
df

,num_groups,pool_size,cumInfections_naive_mean,cumInfections_naive_sem,cumInfections_correlated_mean,cumInfections_correlated_sem,cumInfections_correlated_weak_mean,cumInfections_correlated_weak_sem,cum_num_tests_naive_mean,cum_num_tests_naive_sem,cum_num_tests_correlated_mean,cum_num_tests_correlated_sem,cum_num_tests_correlated_weak_mean,cum_num_tests_correlated_weak_sem
0,5,10,3552.888889,57.264774,2983.0,72.943052,3180.333333,72.36758,47041.111111,391.620208,34572.0,338.111783,39066.888889,427.037663


In [9]:
df.to_csv("../data/aggregated/infections_testconsumption_dilution.csv", index=False)

* Data: aggregate infections for all policies, NP and CCP, and all test error models
* Format: pickled dictionaries

In [10]:
import pickle

NUM_TRIALS = 10

param_config = copy.deepcopy(BASE_PARAM_CONFIG)
param_config["dilute"] = ["average"]

import itertools

param_values = {}

for param in [
    "country",
    "pop_size",
    "init_prev",
    "num_groups",
    "pool_size",
    "horizon",
    "beta",
    "sigma",
    "lamda",
    "gamma",
    "LoD",
    "edge_weight",
    "alpha",
    "peak_VL",
    "distancing_scale",
    "dilute",
]:
    if param in param_config:
        param_values[param] = param_config[param]
    else:
        param_values[param] = [param_config[param + "_default"]]

all_param_configs = [
    dict(zip(param_values.keys(), x)) for x in itertools.product(*param_values.values())
]

In [11]:
import copy
from collections import defaultdict
import numpy as np


final_inf_np = {}
final_inf_cp = {}
final_tests_np = {}
final_tests_cp = {}

for param_config_single in all_param_configs:

    path = f"../data/raw/{param_config_single['country']}"
    for param in [
        "pop_size",
        "init_prev",
        "num_groups",
        "pool_size",
        "horizon",
        "beta",
        "sigma",
        "lamda",
        "gamma",
        "LoD",
        "edge_weight",
        "alpha",
        "peak_VL",
        "distancing_scale",
        "dilute",
    ]:
        path += f"_{param}={param_config_single[param]}"

    results = defaultdict(list)

    for pooling_strategy in ["naive", "correlated_weak"]:
        for i in range(1, NUM_TRIALS + 1):
            filepath = path + f"/{pooling_strategy}/results_{i}.pickle"
            print(f"loading {filepath}")
            try:
                with open(filepath, "rb") as f:
                    results[pooling_strategy].append(pickle.load(f))
            except Exception as e:
                print("error: ", e)
                continue

    if len(results["naive"]) == 0:
        continue

    cumInfections_by_trial = {}
    cumTests_by_trial = {}

    for metric in ["cumInfections", "cum_num_tests"]:

        for pooling_strategy in ["naive", "correlated_weak"]:

            results_tmp = []
            for seed_idx in range(NUM_TRIALS):
                try:
                    results_tmp.append(
                        [x[metric] for x in results[pooling_strategy][seed_idx]]
                    )
                except:
                    pass

            results_tmp_ = np.array(
                [(xi + [xi[-1]] * (100 - len(xi)))[:100] for xi in results_tmp],
                dtype=float,
            )
            if metric == "cumInfections":
                cumInfections_by_trial[pooling_strategy] = results_tmp_[:, -1]
            elif metric == "cum_num_tests":
                cumTests_by_trial[pooling_strategy] = results_tmp_[:, -1]

    final_inf_np[
        (
            param_config_single["num_groups"],
            param_config_single["pool_size"],
            param_config_single["dilute"],
        )
    ] = cumInfections_by_trial["naive"]
    final_inf_cp[
        (
            param_config_single["num_groups"],
            param_config_single["pool_size"],
            param_config_single["dilute"],
        )
    ] = cumInfections_by_trial["correlated_weak"]
    final_tests_np[
        (
            param_config_single["num_groups"],
            param_config_single["pool_size"],
            param_config_single["dilute"],
        )
    ] = cumTests_by_trial["naive"]
    final_tests_cp[
        (
            param_config_single["num_groups"],
            param_config_single["pool_size"],
            param_config_single["dilute"],
        )
    ] = cumTests_by_trial["correlated_weak"]


# save final_inf_np, final_inf_cp
with open("../data/aggregated/infections_np_nodil.pickle", "wb") as f:
    pickle.dump(final_inf_np, f)
with open("../data/aggregated/infections_ccp_nodil.pickle", "wb") as f:
    pickle.dump(final_inf_cp, f)

with open("../data/aggregated/testconsumption_np_nodil.pickle", "wb") as f:
    pickle.dump(final_tests_np, f)
with open("../data/aggregated/testconsumption_ccp_nodil.pickle", "wb") as f:
    pickle.dump(final_tests_cp, f)

loading ../data/raw/US_pop_size=10000_init_prev=0.01_num_groups=5_pool_size=10_horizon=100_beta=0.1_sigma=0.2_lamda=0.5_gamma=0.25_LoD=1240_edge_weight=10_alpha=2.0_peak_VL=6.0_distancing_scale=50.0_dilute=average/naive/results_1.pickle
loading ../data/raw/US_pop_size=10000_init_prev=0.01_num_groups=5_pool_size=10_horizon=100_beta=0.1_sigma=0.2_lamda=0.5_gamma=0.25_LoD=1240_edge_weight=10_alpha=2.0_peak_VL=6.0_distancing_scale=50.0_dilute=average/naive/results_2.pickle
loading ../data/raw/US_pop_size=10000_init_prev=0.01_num_groups=5_pool_size=10_horizon=100_beta=0.1_sigma=0.2_lamda=0.5_gamma=0.25_LoD=1240_edge_weight=10_alpha=2.0_peak_VL=6.0_distancing_scale=50.0_dilute=average/naive/results_3.pickle
loading ../data/raw/US_pop_size=10000_init_prev=0.01_num_groups=5_pool_size=10_horizon=100_beta=0.1_sigma=0.2_lamda=0.5_gamma=0.25_LoD=1240_edge_weight=10_alpha=2.0_peak_VL=6.0_distancing_scale=50.0_dilute=average/naive/results_4.pickle
loading ../data/raw/US_pop_size=10000_init_prev=0.01

* Data: detailed trajectory info for (5,10) under the dilution model
* Format: pickled nested list-dictionary

In [12]:
import pickle

param_config = copy.deepcopy(BASE_PARAM_CONFIG)
param_config["dilute"] = ["average"]

import itertools

param_values = {}

for param in [
    "country",
    "pop_size",
    "init_prev",
    "num_groups",
    "pool_size",
    "horizon",
    "beta",
    "sigma",
    "lamda",
    "gamma",
    "LoD",
    "edge_weight",
    "alpha",
    "peak_VL",
    "distancing_scale",
    "dilute",
]:
    if param in param_config:
        param_values[param] = param_config[param]
    else:
        param_values[param] = [param_config[param + "_default"]]

all_param_configs = [
    dict(zip(param_values.keys(), x)) for x in itertools.product(*param_values.values())
]

In [13]:
import copy
from collections import defaultdict


trajectories = (
    []
)  # list of tuples (dict of param_config_single, dict of {metric: [mean, std]})

for param_config_single in all_param_configs:

    traj_info = [param_config_single, {}]

    path = f"../data/raw/{param_config_single['country']}"
    for param in [
        "pop_size",
        "init_prev",
        "num_groups",
        "pool_size",
        "horizon",
        "beta",
        "sigma",
        "lamda",
        "gamma",
        "LoD",
        "edge_weight",
        "alpha",
        "peak_VL",
        "distancing_scale",
        "dilute",
    ]:
        path += f"_{param}={param_config_single[param]}"

    results = defaultdict(list)

    for pooling_strategy in ["naive", "correlated", "correlated_weak"]:
        for i in range(1, NUM_TRIALS + 1):
            filepath = path + f"/{pooling_strategy}/results_{i}.pickle"
            print(f"loading {filepath}")
            try:
                with open(filepath, "rb") as f:
                    results[pooling_strategy].append(pickle.load(f))
            except Exception as e:
                # print("error: ", e)
                continue

    if len(results["naive"]) == 0:
        continue

    metrics = set(results["correlated"][0][0].keys()) - set(
        ["day", "VL_in_positive_pools"]
    )

    for metric in metrics:

        for pooling_strategy in ["naive", "correlated", "correlated_weak"]:

            results_tmp = []
            for SEED in range(NUM_TRIALS):
                try:
                    results_tmp.append(
                        [x[metric] for x in results[pooling_strategy][SEED]]
                    )
                except:
                    pass

            results_tmp_ = np.array(
                [(xi + [xi[-1]] * (100 - len(xi)))[:100] for xi in results_tmp],
                dtype=float,
            )
            mean = np.nanmean(results_tmp_, axis=0)
            sem = np.nanstd(results_tmp_, axis=0) / np.sqrt(NUM_TRIALS)
            traj_info[1][metric + "_" + pooling_strategy] = [mean, sem]

    trajectories.append(traj_info)

with open(
    "../data/aggregated/trajectory_info_numgroups=5_poolsize=10.pickle", "wb"
) as f:
    pickle.dump(trajectories, f)

loading ../data/raw/US_pop_size=10000_init_prev=0.01_num_groups=5_pool_size=10_horizon=100_beta=0.1_sigma=0.2_lamda=0.5_gamma=0.25_LoD=1240_edge_weight=10_alpha=2.0_peak_VL=6.0_distancing_scale=50.0_dilute=average/naive/results_1.pickle
loading ../data/raw/US_pop_size=10000_init_prev=0.01_num_groups=5_pool_size=10_horizon=100_beta=0.1_sigma=0.2_lamda=0.5_gamma=0.25_LoD=1240_edge_weight=10_alpha=2.0_peak_VL=6.0_distancing_scale=50.0_dilute=average/naive/results_2.pickle
loading ../data/raw/US_pop_size=10000_init_prev=0.01_num_groups=5_pool_size=10_horizon=100_beta=0.1_sigma=0.2_lamda=0.5_gamma=0.25_LoD=1240_edge_weight=10_alpha=2.0_peak_VL=6.0_distancing_scale=50.0_dilute=average/naive/results_3.pickle
loading ../data/raw/US_pop_size=10000_init_prev=0.01_num_groups=5_pool_size=10_horizon=100_beta=0.1_sigma=0.2_lamda=0.5_gamma=0.25_LoD=1240_edge_weight=10_alpha=2.0_peak_VL=6.0_distancing_scale=50.0_dilute=average/naive/results_4.pickle
loading ../data/raw/US_pop_size=10000_init_prev=0.01